# Welcome to Semantic Kernel with Microsoft Fabric - Getting Started
- This notebook has following examples:
    - install semantic-kernel library in Microsoft Fabric Spark environment
    - Run prompts inline, directly in the code
    - Run prompts from a file in the LakeHouse
- Todo: Secrets are hardcoded in this notebook for simplicity for the workshop - to move to Azure Keyvault
- Prerequisites:
    - Access to Azure OpenAI, and values for the endpoint, the key, the deployments
    - Upload Skills to the Lakehouse (Todo - Write instructions for how to do this)

In [2]:
%pip install semantic-kernel --quiet
%pip show semantic-kernel

StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 11, Finished, Available)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lime 0.2.0.1 requires scikit-image>=0.12, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.2 which is incompatible.

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: /nfs4/pyenv-eeed4a44-3a92-4b49-a275-6ddc8a6428a4/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Name: semantic-kernel
Version: 0.3.8.dev0
Summary: 
Home-page: 
Author: Microsoft
Author-email: SK-Support@microsoft.com
License: 
Location: /nfs4/pyenv-eeed4a44-3a92-4b49-a275-6ddc8a6428a4/lib/python3.10/site-packages
Requires: aiofiles, numpy, openai, openapi_core, prance, pydantic, python-dotenv, regex
Required-by: 
Note: you may need to restart the kernel to use updated packages.



In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion

StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 13, Finished, Available)

In [18]:
deployment = "text-davinci-003"    ###"gpt-35-turbo" 
api_key = "********"
endpoint = "https://******.openai.azure.com/"
#print(deployment, api_key, endpoint)

StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 59, Finished, Available)

In [19]:
kernel = sk.Kernel()
kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))


StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 60, Finished, Available)

In [20]:
# Wrap your prompt in a function
prompt = kernel.create_semantic_function("""
Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place.

With Fabric, you don't need to piece together different services from multiple vendors. Instead, you can enjoy a highly integrated, end-to-end, and easy-to-use product that is designed to simplify your analytics needs.

The platform is built on a foundation of Software as a Service (SaaS), which takes simplicity and integration to a whole new level.

Microsoft Fabric brings together new and existing components from Power BI, Azure Synapse, and Azure Data Factory into a single integrated environment. These components are then presented in various customized user experiences.

Write a slogan or sales pitch.
""",
        max_tokens=512,
        temperature=0.0,
        top_p=0.0,
        presence_penalty=0.0,
        frequency_penalty=0.0,)

StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 61, Finished, Available)

In [21]:
# Run your prompt
print(prompt()) 

StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 62, Finished, Available)


"Unlock the power of analytics with Microsoft Fabric - the all-in-one solution for data movement, data science, real-time analytics, and business intelligence!"


In [22]:
prompt = """
Generate a short funny poem or limerick to explain the given event. Be creative and be funny. Let your imagination run wild.

+++++

Event:{{$input}}
+++++
"""

StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 63, Finished, Available)

In [23]:
promptFunction = kernel.create_semantic_function(
        prompt,
        max_tokens=100,
        temperature=0.9,
        top_p=0.0,
        presence_penalty=0.0,
        frequency_penalty=0.0,
    )

StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 65, Finished, Available)

In [29]:
input_text = "Friday"
completions = await promptFunction.invoke_async(input_text)

print(completions)

StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 84, Finished, Available)


On Friday, I'm so glad,
It's the start of the weekend, not bad!
I'll take a break,
From the work I make,
And enjoy the weekend I've had!


In [25]:
# note: using skills from the samples folder
skills_directory = "/lakehouse/default/Files/skills"

SummarizeFunction = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")["Summarize"]



StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 70, Finished, Available)

In [26]:
SampleText = """Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place.

With Fabric, you don't need to piece together different services from multiple vendors. Instead, you can enjoy a highly integrated, end-to-end, and easy-to-use product that is designed to simplify your analytics needs.

The platform is built on a foundation of Software as a Service (SaaS), which takes simplicity and integration to a whole new level.

Microsoft Fabric brings together new and existing components from Power BI, Azure Synapse, and Azure Data Factory into a single integrated environment. These components are then presented in various customized user experiences.

"""

StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 74, Finished, Available)

In [27]:
print(SampleText)

StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 75, Finished, Available)

Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place.

With Fabric, you don't need to piece together different services from multiple vendors. Instead, you can enjoy a highly integrated, end-to-end, and easy-to-use product that is designed to simplify your analytics needs.

The platform is built on a foundation of Software as a Service (SaaS), which takes simplicity and integration to a whole new level.

Microsoft Fabric brings together new and existing components from Power BI, Azure Synapse, and Azure Data Factory into a single integrated environment. These components are then presented in various customized user experiences.




In [28]:
result = SummarizeFunction(SampleText)

print(result)

# You can also invoke functions asynchronously
# result = await jokeFunction.invoke_async("time travel to dinosaur age")
# print(result)

StatementMeta(, a1c24dd6-8e38-4bf7-a272-fda417861457, 76, Finished, Available)


Microsoft Fabric is an all-in-one analytics solution that provides data movement, data science, real-time analytics, and business intelligence services. It is built on a foundation of Software as a Service (SaaS) and integrates components from Power BI, Azure Synapse, and Azure Data Factory into a single environment with customized user experiences.


##### End of this Notebook